# 아래 [실행] 부분에 이 이모지(📌) 있는 부분 코드만 바꿔서 전체 코드 돌리시면 됩니당 

In [6]:
import requests
import json
import pandas as pd
from datetime import datetime
import time

[함수 1] (입력) 역 이름 → (출력) 역 코드

In [7]:
def station_name_to_code(station_name): # 입력 형식: string (station_NM = "신촌") -> 출력 형식: string(return = '0240')
    station_dict = {
        '한양대': '0209', '신도림': '0234', '아현': '0242', '봉천': '0229', '충정로': '0243', 
        '용답': '0244', '문래': '0235', '잠실새내': '0217', '신림': '0230', '종합운동장': '0218', 
        '삼성': '0219', '신대방': '0231', '영등포구청': '0236', '양천구청': '0248', '을지로4가': '0204', 
        '뚝섬': '0210', '신답': '0245', '을지로입구': '0202', '사당': '0226', '을지로3가': '0203', 
        '성수': '0211', '건대입구': '0212', '당산': '0237', '신정네거리': '0249', '구의': '0213', 
        '합정': '0238', '용두': '0250', '구로디지털단지': '0232', '홍대입구': '0239', '신촌': '0240', 
        '이대': '0241', '신설동': '0246', '도림천': '0247', '까치산': '0200', '시청': '0201', 
        '동대문역사문화공원': '0205', '신당': '0206', '상왕십리': '0207', '강변': '0214', '잠실나루': '0215', 
        '잠실': '0216', '선릉': '0220', '역삼': '0221', '강남': '0222', '교대': '0223', '서초': '0224', 
        '방배': '0225', '낙성대': '0227', '서울대입구': '0228', '왕십리': '0208', '대림': '0233'
    }

    if station_name not in station_dict:
        print("해당 역이 존재하지 않습니다.")
        return -1
        
    return station_dict[station_name]

[함수 2] (입력) 역 이름, 시간, 상행(1)/하행(2) + SEOUL_KEY → (출력) 현재 열차 코드

In [8]:
def train_code(STATION_NAME, TIME, INOUT_TAG, SEOUL_KEY):
    
    STATION_CD = station_name_to_code(STATION_NAME)
    if (STATION_CD == -1): 
        return -1
    
    # 입력값 -> 수정 X
    SERVICE = "SearchSTNTimeTableByIDService"
    START_INDEX = 0 # 페이징 시작번호: 데이터 행 시작번호
    END_INDEX = 200 # 페이징 끝번호 : 데이터 행 끝번호
    WEEK_TAG = 1 # 평일:1, 토요일:2, 휴일/일요일:3
    TYPE = "json"
    
    # 호출 url
    url = f"http://openAPI.seoul.go.kr:8088/{SEOUL_KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{STATION_CD}/{WEEK_TAG}/{INOUT_TAG}"
    response = requests.get(url)

    # 시간에 따른 열차 번호 찾기(주어진 시간 직후의 열차 번호) 
    #  입력값: 시간
    
    # JSON 데이터를 딕셔너리로 파싱
    parsed_data = json.loads(response.text)
    
     # "row" 키에 해당하는 값에서 "ARRIVETIME"이 TIME보다 큰 빠른 열차 번호 찾기
    fast_train = None
    min_time = float('inf')
    for train in parsed_data["SearchSTNTimeTableByIDService"]["row"]:
        if train.get("ARRIVETIME") >= TIME:
            train_time = int(train.get("ARRIVETIME").replace(":", ""))
            if train_time < min_time:
                min_time = train_time
                fast_train = train["TRAIN_NO"]

    if fast_train == None:
        print("해당 열차가 존재하지 않습니다.")
        return -1
    
    return fast_train
        

[함수 3] (입력) 열차 번호 + SK_KEY -> (출력) 실시간 혼잡도

In [9]:
def train_number_to_current_congestion_list(TRAIN_NUMBER, SK_KEY):
    url = f"https://apis.openapi.sk.com/puzzle/subway/congestion/rltm/trains/2/{TRAIN_NUMBER}"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "appkey": SK_KEY}
    
    response = requests.get(url, headers=headers)
    result_response = json.loads(response.text)

    if result_response["success"] is False:
        print("해당 열차에 대한 실시간 혼잡도 정보가 존재하지 않습니다.")
        return -1

    
    # JSON 문자열을 파이썬 객체로 변환
    parsed_data = json.loads(response.text)
    
    # "congestionCar" 값을 파이썬 리스트로 변환
    congestion_car_list = parsed_data['data']['congestionResult']['congestionCar'].split('|')
    
    return congestion_car_list

[함수 4] (입력) 역 이름, 상행(1)/하행(2) + SEOUL_KEY, SK_KEY,  -> (출력) 그 역에 현재 오는 열차의 실시간 혼잡도

In [10]:
def current_train_congestion_list(STATION_NAME, INOUT_TAG, SEOUL_KEY, SK_KEY):
    
    # 현재 시간 받아오기
    CURRENT_TIME = datetime.now().strftime('%H:%M:%S')
    
    # 현재 시간에 <STATION_NAME>에 들어오는 <INOUT_TAG>행 열차 번호 받아오기
    TRAIN_NUMBER = train_code(STATION_NAME, CURRENT_TIME, INOUT_TAG, SEOUL_KEY)
    # print("TRAIN_NUMBER: " + str(TRAIN_NUMBER))

    if TRAIN_NUMBER == -1:
        return -1

    # 그 열차에 대한 실시간 혼잡도 리스트 출력
    CONGESTION_LIST = train_number_to_current_congestion_list(TRAIN_NUMBER, SK_KEY)
    
    if CONGESTION_LIST == -1:
        return -1
        
    return CONGESTION_LIST

[실행]

**1. 📌 이모지가 있는 역 이름, 상행/하행, 인증키 두 개 변경하고 코드 돌리시면 됩니다.**

**2. 6:00부터 10:50까지 총 30번 돌아가야 해서 for문의 range가 30인데, 그 시간을 줄이시고 싶으면 range() 안의 숫자를 바꿔주세요.**

    예를 들어서, 6:00~6:50만 수집하고 싶으면 range(6)으로 바꾸고 6시에 실행시켜 주시면 됩니다.
   
**3. 한 번 돌릴 때마다 api를 30번 쓰기 때문에, 하루에 3번까지만 돌리실 수 있습니다.**
   

In [12]:
STATION_NAME = '을지로3가' #📌 역 이름(문자열 형식)
INOUT_TAG = 1 #📌 상행,내선: 1 / 하행,외선: 2
SEOUL_KEY = "486d5742676f746136314f63476259" #📌 서울 열린 데이터 광장 인증키
SK_KEY = "DWJ3uJdzMy5lKNdoDgNUa2fe7zUjC3zw3UlfZ5h1" #📌 SK open API 앱키

result_data = []
# 여기서부터 반복
for i in range(30): # 6:00 ~ 10:50 (📌 시간을 더 짧게 줄이고 싶으면, range안에 있는 숫자 변경)
    CURRENT_TIME = datetime.now().strftime('%H:%M:%S')
    CONGESTION_LIST = current_train_congestion_list(STATION_NAME, INOUT_TAG, SEOUL_KEY, SK_KEY)
    
    if (CONGESTION_LIST == -1):
        print("데이터 수집 실패")
    else:
        Hour, Minute, Second = CURRENT_TIME.split(":")
        data = {
            'Station Name': STATION_NAME,
            'Up/Down': INOUT_TAG,
            'Hour': Hour,
            'Minute': Minute,
            'Congestion List': CONGESTION_LIST
        }
        result_data.append(data)
        
    time.sleep(600) # 10분 멈춤

# 딕셔너리를 데이터프레임으로 변환
df2 = pd.DataFrame(result_data)

df2

ConnectionError: HTTPConnectionPool(host='openapi.seoul.go.kr', port=8088): Max retries exceeded with url: /486d5742676f746136314f63476259/json/SearchSTNTimeTableByIDService/0/200/0203/1/1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000284FD40F3D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [13]:
df2

NameError: name 'df2' is not defined

[파일로 저장]

In [18]:
# 데이터 프레임을 파일로 저장

if INOUT_TAG == 1:
    UPDOWN = 'UP'
elif INOUT_TAG == 2:
    UPDOWN = 'DOWN'
df2.to_csv(STATION_NAME + "_" + UPDOWN + "_" + "실시간_혼잡도.csv")